<a href="https://colab.research.google.com/github/ari-jakubowicz/dataportfolio/blob/main/media_optimizers_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [976 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ge

In [ ]:
!apt install -y mysql-server mysql-client

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl libfcgi-bin libfcgi-perl
  libfcgi0ldbl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2 libprotobuf-lite23
  liburi-perl mecab-ipadic mecab-ipadic-utf8 mecab-utils mysql-client-8.0 mysql-client-core-8.0
  mysql-server-8.0 mysql-server-core-8.0
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libbusiness-isbn-perl libwww-perl mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl libfcgi-bin libfcgi-perl
  libfcgi0ldbl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2 libprotobuf-l

In [ ]:
!pip install mysql-connector-python
import mysql.connector


mydb=mysql.connector.connect(
    host="mfm.cacmdzhzwi3c.us-west-1.rds.amazonaws.com",
    user="mo_test_user",
    password="motestuser",
    database="mo_test_schema"
)

mycursor = mydb.cursor()

mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 68.7 MB/s eta 0:00:00
('AD_Total_Spent',)
('EndTable',)
('ad_spent',)
('ad_spent_view',)
('contacts',)
('data',)
('data1',)
('detail',)
('filtered_preview',)
('final_stats',)
('latest_event',)
('latest_updates',)
('market_group',)
('metrics',)
('part1',)
('preview',)
('preview_categorization',)
('processed_data5',)
('view',)


In [ ]:
import pandas as pd


myquery = '''
select distinct
  p.contact_id,
  p.id as event_id,
  mg.market_group,
  p.tags,
  p.last_update as prev_last_upd,
  p.source
from preview p
inner join market_group mg on mg.id = p.market_group_id

'''

mycursor.execute(myquery)


column_names = [i[0] for i in mycursor.description]

# Cargar los resultados en un DataFrame de Pandas
result = mycursor.fetchall()
df = pd.DataFrame(result, columns=column_names)


df['prev_last_upd'] = pd.to_datetime(df['prev_last_upd'])


# Select the latest registration for each contact within each market group
df = df.sort_values('prev_last_upd').groupby(['contact_id', 'market_group']).tail(1)

# Filter and keep the GUESTS
guests_filtered_df = df[df['tags'].str.contains('preview_guest') | df['tags'].str.contains('partner')]

# COUNT GUESTS Perform the same grouping on the filtered data for guest counts and other aggregations
grouped_df = guests_filtered_df.groupby(['market_group', 'source']).agg({
    'contact_id': 'count',     # Take the count of contact_id (i.e., guest count)
}).reset_index()
grouped_df.rename(columns={'contact_id': 'guests_count'}, inplace=True)

# Filter and keep REGISTRATIONS (rows that do not contain 'preview_guest' or 'partner')
registration_filtered_df = df[~df['tags'].str.contains('preview_guest') & ~df['tags'].str.contains('partner')]

# COUNT REGISTRATIONS Group by 'market_group' and 'source' and count the number of registrations
registrations_counts = registration_filtered_df.groupby(['market_group', 'source']).agg({
    'contact_id': 'count',  # Count non-NaN contact_id values for registrations
}).reset_index()
registrations_counts.rename(columns={'contact_id': 'registrations_count'}, inplace=True)

#Filter and keep the ATTENDEES
attendee_filtered_df = df[df['tags'].str.contains('pb') | df['tags'].str.contains('pnb')]

# COUNT ATTENDEES Group by 'market_group' and 'source' and count the attendees
attendees_counts = attendee_filtered_df.groupby(['market_group', 'source']).agg({
    'contact_id': 'count'  # Count non-NaN contact_id values for attendees
}).reset_index()
attendees_counts.rename(columns={'contact_id': 'attendees_count'}, inplace=True)

# Filter and keep for buyers
buyer_filtered_df = df[df['tags'].str.contains('pb')]

# COUNT BUYERS Group by 'market_group' and 'source' and count the buyers
buyers_counts = buyer_filtered_df.groupby(['market_group', 'source']).agg({
    'contact_id': 'count'  # Count non-NaN contact_id values for buyers
}).reset_index()
buyers_counts.rename(columns={'contact_id': 'buyers_count'}, inplace=True)

# Merge the registration counts with the grouped data which already includes guests
grouped_df = pd.merge(grouped_df, registrations_counts, on=['market_group', 'source'], how='left')
grouped_df = pd.merge(grouped_df, attendees_counts, on=['market_group', 'source'], how='left')
grouped_df = pd.merge(grouped_df, buyers_counts, on=['market_group', 'source'], how='left')

grouped_df['source'] = grouped_df['source'].replace('', 'empty source')


grouped_df

,market_group,source,guests_count,registrations_count,attendees_count,buyers_count
0,20231023_ANC,empty source,82,11.0,2,NaN
1,20231023_ANC,fb,2,123.0,28,14.0
2,20231023_ANC,fbla,3,341.0,38,6.0
3,20231023_ANC,gs,2,31.0,9,2.0
4,20231023_ANC,in-person,1,7.0,7,6.0
5,20231023_ANC,yt,13,177.0,48,15.0
6,20231023_MNH,empty source,180,23.0,9,5.0
7,20231023_MNH,direct,4,21.0,9,4.0
8,20231023_MNH,fb,11,394.0,74,14.0
9,20231023_MNH,fbla,7,659.0,51,14.0


In [ ]:
import pandas as pd


ads_myquery = '''
select distinct
  p.contact_id,
  p.id as event_id,
  ad.market_group,
  p.tags,
  p.last_update as prev_last_upd,
  p.source,
  ad.amount_spent
from preview p
left join ad_spent ad on ad.source = p.source

'''

mycursor.execute(ads_myquery)


column_names = [i[0] for i in mycursor.description]

# Cargar los resultados en un DataFrame de Pandas
result = mycursor.fetchall()
df = pd.DataFrame(result, columns=column_names)

df = df.groupby(['source', 'market_group'])['amount_spent'].sum().reset_index()
df = df.sort_values(by='market_group').reset_index(drop=True)
df

,source,market_group,amount_spent
0,fb,20231023_ANC,28625903.67
1,gs,20231023_ANC,309820.66
2,yt,20231023_ANC,75291482.38
3,fbla,20231023_ANC,35447881.36
4,gs,20231023_MNH,451613.64
5,fbla,20231023_MNH,45268555.44
6,yt,20231023_MNH,104198261.60
7,fb,20231023_MNH,42881196.54
8,fbla,20231023_PHL,41779159.36
9,fb,20231023_PHL,40118981.76


In [ ]:
import pandas as pd


myquery = '''
select distinct
  market_group,
  source,
  amount_spent
from ad_spent
limit 50000

'''

mycursor.execute(myquery)


column_names = [i[0] for i in mycursor.description]

# Cargar los resultados en un DataFrame de Pandas
result = mycursor.fetchall()
df = pd.DataFrame(result, columns=column_names)

df = df.groupby(['market_group', 'source'])['amount_spent'].sum().reset_index()

df = df.merge(grouped_df, on=['market_group', 'source'], how='right')

#Convert amount spent to float
df['amount_spent'] = df['amount_spent'].astype(float)

df['cost_per_registration'] = df['amount_spent'] / df['registrations_count']
df['cost_per_attendee'] = df['amount_spent'] / df['attendees_count']
df['cost_per_buyer'] = df['amount_spent'] / df['buyers_count']
df.fillna(0, inplace=True)
df

,market_group,source,amount_spent,guests_count,registrations_count,attendees_count,buyers_count,cost_per_registration,cost_per_attendee,cost_per_buyer
0,20231023_ANC,empty source,0.00,82,11.0,2,0.0,0.000000,0.000000,0.000000
1,20231023_ANC,fb,13849.01,2,123.0,28,14.0,112.593577,494.607500,989.215000
2,20231023_ANC,fbla,9695.81,3,341.0,38,6.0,28.433460,255.152895,1615.968333
3,20231023_ANC,gs,986.69,2,31.0,9,2.0,31.828710,109.632222,493.345000
4,20231023_ANC,in-person,0.00,1,7.0,7,6.0,0.000000,0.000000,0.000000
5,20231023_ANC,yt,25214.83,13,177.0,48,15.0,142.456667,525.308958,1680.988667
6,20231023_MNH,empty source,0.00,180,23.0,9,5.0,0.000000,0.000000,0.000000
7,20231023_MNH,direct,0.00,4,21.0,9,4.0,0.000000,0.000000,0.000000
8,20231023_MNH,fb,20745.62,11,394.0,74,14.0,52.653858,280.346216,1481.830000
9,20231023_MNH,fbla,12381.99,7,659.0,51,14.0,18.789059,242.784118,884.427857


In [ ]:
## CREATE TABLE FROM DF



import mysql.connector

# Assuming you already have a connection and cursor
# connection = mysql.connector.connect( ... )
# cursor = connection.cursor()

# Define the SQL query to create a table
create_table_query = '''
CREATE TABLE IF NOT EXISTS event_stats (
    market_group VARCHAR(255),
    source VARCHAR(255),
    amount_spent FLOAT,
    guests_count INT,
    registrations_count FLOAT,
    attendees_count INT,
    buyers_count FLOAT,
    cost_per_registration FLOAT,
    cost_per_attendee FLOAT,
    cost_per_buyer FLOAT
);
'''

# Execute the query
mycursor.execute(create_table_query)
mydb.commit()

###########


# Sample DataFrame
# Convert the DataFrame to a list of tuples
data = list(df.itertuples(index=False, name=None))

# Prepare the insert query
insert_query = '''
INSERT INTO event_stats (
    market_group, source, amount_spent, guests_count,
    registrations_count, attendees_count, buyers_count,
    cost_per_registration, cost_per_attendee, cost_per_buyer
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
'''

# Insert each row from DataFrame into the table
mycursor.executemany(insert_query, data)

# Commit the transaction
mydb.commit()